Copyright &copy; 2017 D.S. Parker

# Final Exam Problem: Analyzing The Analyzers


The goal of this problem is to define "Data Scientists", based on the StackOverflow Tech Job Survey,
using nonnegative matrix factorization.

This is the method used in the O'Reilly booklet <b>Analyzing the Analyzers</b>
discussed at the start of this course.

For this problem,
however, you are asked to use the Stackoverflow data that you used in HW6,
rather than the survey the authors used.

(Stackoverflow does a large survey of tech workers on jobs:
https://insights.stackoverflow.com/survey )

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from patsy import dmatrices

import pandas as pd
from pandas import Series, DataFrame

import statsmodels.api as sm

from sklearn import datasets, svm

## Part 0:  Getting the Survey data

First, go to the survey site and download the 2016 data:
https://insights.stackoverflow.com/survey
    
(The results of the 2017 survey are there, but the data is not yet available for download now.)

# IMPORTANT

For this problem, we want you to extract a subset of the data involving <i>data science</i>,
and compare the results for it with results for the entire dataset.

The <b>occupation</b> column in the dataset (i.e., column number 10)
involves job titles.  Please extract all rows that have
any of the job titles of interest here
(with the number of occurrences listed first):
<ul><li>
<b>Business intelligence or data warehousing expert</b> (392 occurrences)
</li><li>
<b>Data scientist</b> (800 occurrences)
</li><li>
<b>Developer with a statistics or mathematics background</b>  (960)
</li><li>
<b>Machine learning developer</b> (385).
</li></ul>

For this problem, extract all rows with these job titles into a <tt>DataScience</tt> table with about 2500 rows.

<!--
We also want you to consider the entire dataset; for simplicity let's call it <tt>JobSurvey</tt>.
-->

In [4]:
JobSurvey = pd.read_csv('2016 Stack Overflow Survey Responses.csv')

JobSurvey.shape

(56030, 66)

#  IN THIS ASSIGNMENT, DROP MISSING VALUES CAREFULLY

Instead of dropping all rows with nulls (a drastic solution), first selectively delete columns in a way that maximizes the size of the "data scientist" dataset.


## A nice way to avoid missing values:  some missing values are not 'NA's

Your alert classmate Phillip David discovered that the 'star_wars_vs_star_trek' column
was based on a question that permitted '' (empty string) as an answer.
This gets turned into NaN by Pandas when reading in data.

So this column should have something like '' instead of NaN
(after Pandas reads in the data).
Thus we can use .fillna('') to replace all the NaNs in this column.

In fact, Phillip showed that lots of columns can get fixed this way:
<pre>
self_identification,
education,
tech_do,
tech_want,
how_to_improve_interview_process,
why_stack_overflow,
dev_environment,
developer_challenges,
star_wars_vs_star_trek,
new_job_value
</pre>

The code below repairs each of these columns.

In [5]:
JobSurvey['star_wars_vs_star_trek'].head()

0          NaN
1    Star Wars
2          NaN
3          NaN
4          NaN
Name: star_wars_vs_star_trek, dtype: object

In [6]:
JobSurvey['star_wars_vs_star_trek'] = JobSurvey['star_wars_vs_star_trek'].fillna('')

JobSurvey['star_wars_vs_star_trek'].head()

0             
1    Star Wars
2             
3             
4             
Name: star_wars_vs_star_trek, dtype: object

In [7]:
Phillips_repairable_columns = 'self_identification,education,tech_do,tech_want,\
how_to_improve_interview_process,why_stack_overflow,dev_environment,\
developer_challenges,star_wars_vs_star_trek,new_job_value'.split(',')
#  the .split(',') method turns this into a list of strings

for column in Phillips_repairable_columns:
    JobSurvey[column] = JobSurvey[column].fillna('')
    
JobSurvey[Phillips_repairable_columns].head()

,self_identification,education,tech_do,tech_want,how_to_improve_interview_process,why_stack_overflow,dev_environment,developer_challenges,star_wars_vs_star_trek,new_job_value
0,Programmer,,,,,,,,,
1,Developer; Engineer; Programmer,I'm self-taught; On-the-job training; B.S. in ...,iOS; Objective-C,Swift,Show me more live code; Introduce me to the te...,To get help for my job; Because I can't do my ...,TextMate; Xcode,Unrealistic expectations; Interacting with non...,Star Wars,Salary; Ability to make or influence important...
2,,,,,,,,,,
3,Engineer,I'm self-taught; On-the-job training,,,Be more flexible about interview scheduling; O...,To get help for my job; Beacause I love to learn,Visual Studio,Corporate policies; Interacting with non-techn...,,Office location; Remote working option; Flexib...
4,Developer; Engineer; Programmer; Sr. Developer...,,,,,,,,,


##  Another column that can be fixed:  The Big Mac Index

The 'Big Mac Index' is the Jan 2016 local price of a Big Mac divided by dollar price,
and is inferrable from Country.  See:  http://www.economist.com/content/big-mac-index

The index is only tracked by the Economist in 56 countries, and so is not always entered.
Since it is undefined mainly by poor countries, we just fillna(...) with the highest value.

In [8]:
max_big_mac_index = JobSurvey['big_mac_index'].max()
max_big_mac_index

6.4400000000000004

In [9]:
JobSurvey['big_mac_index'] = JobSurvey['big_mac_index'].fillna(max_big_mac_index)

In [10]:
JobSurvey['big_mac_index'].head()

0    6.44
1    6.44
2    6.44
3    6.44
4    6.44
Name: big_mac_index, dtype: float64

## Taking Stock:  Find out how many values are non-null in each column

In [11]:
JobSurvey.count()

Unnamed: 0                          56030
collector                           56030
country                             55528
un_subregion                        55313
so_region                           55390
age_range                           55727
age_midpoint                        55336
gender                              55586
self_identification                 56030
occupation                          49519
occupation_group                    46934
experience_range                    49520
experience_midpoint                 49520
salary_range                        46121
salary_midpoint                     41742
big_mac_index                       56030
tech_do                             56030
tech_want                           56030
aliens                              49177
programming_ability                 46982
employment_status                   49576
industry                            40110
company_size_range                  39932
team_size_range                   

## Taking Stock for "data scientists": determining which columns have the most missing values

With this information, we can omit columns that have the most missing values (below).

In [12]:
JobSurvey[ JobSurvey.occupation == 'Data scientist' ].count()

Unnamed: 0                          800
collector                           800
country                             796
un_subregion                        795
so_region                           795
age_range                           796
age_midpoint                        786
gender                              797
self_identification                 800
occupation                          800
occupation_group                    800
experience_range                    795
experience_midpoint                 795
salary_range                        735
salary_midpoint                     686
big_mac_index                       800
tech_do                             800
tech_want                           800
aliens                              791
programming_ability                 746
employment_status                   797
industry                            677
company_size_range                  666
team_size_range                     666
women_on_team                       665


In [13]:
JobSurvey[ JobSurvey.occupation == 'Data scientist' ][0:4]

,Unnamed: 0,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,...,important_sameend,important_newtech,important_buildnew,important_buildexisting,important_promotion,important_companymission,important_wfh,important_ownoffice,developer_challenges,why_stack_overflow
28,34171,Facebook,Andorra,Southern Europe,Western Europe,20-24,22.0,Female,Hacker,Data scientist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
34,29315,Facebook,Argentina,South America,South America,20-24,22.0,Male,Developer; Engineer; Sr. Developer; Full-stack...,Data scientist,...,This is somewhat important,This is somewhat important,This is very important,I don't care about this,This is very important,This is somewhat important,This is somewhat important,I don't care about this,Fragile code base; Poor documentation; Changin...,To get help for my job; To give help to others
205,31023,Facebook,Australia,Australia and New Zealand,Oceania,35-39,37.0,Female,Programmer,Data scientist,...,I don't care about this,This is very important,This is somewhat important,This is somewhat important,This is very important,This is very important,This is somewhat important,This is somewhat important,Poor team dynamics; Inefficient development pr...,To get help for my job
225,49442,Facebook,Australia,Australia and New Zealand,Oceania,20-24,22.0,Male,Developer; Engineer; Programmer,Data scientist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,


In [14]:
JobSurvey.columns[0]  # strangely named because the column name was omitted

'Unnamed: 0'

In [15]:
JobSurvey.shape

(56030, 66)

In [16]:
JobSurvey[['jobId']] = JobSurvey[['Unnamed: 0']]
JobSurvey.shape

(56030, 67)

In [17]:
DataScienceJobSurvey = JobSurvey[ JobSurvey.occupation == 'Data scientist' ]
DataScienceJobSurvey[0:5]

,Unnamed: 0,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,...,important_newtech,important_buildnew,important_buildexisting,important_promotion,important_companymission,important_wfh,important_ownoffice,developer_challenges,why_stack_overflow,jobId
28,34171,Facebook,Andorra,Southern Europe,Western Europe,20-24,22.0,Female,Hacker,Data scientist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,34171
34,29315,Facebook,Argentina,South America,South America,20-24,22.0,Male,Developer; Engineer; Sr. Developer; Full-stack...,Data scientist,...,This is somewhat important,This is very important,I don't care about this,This is very important,This is somewhat important,This is somewhat important,I don't care about this,Fragile code base; Poor documentation; Changin...,To get help for my job; To give help to others,29315
205,31023,Facebook,Australia,Australia and New Zealand,Oceania,35-39,37.0,Female,Programmer,Data scientist,...,This is very important,This is somewhat important,This is somewhat important,This is very important,This is very important,This is somewhat important,This is somewhat important,Poor team dynamics; Inefficient development pr...,To get help for my job,31023
225,49442,Facebook,Australia,Australia and New Zealand,Oceania,20-24,22.0,Male,Developer; Engineer; Programmer,Data scientist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,49442
241,2229,Facebook,Austria,Western Europe,Western Europe,50-59,54.5,Male,Engineer; Programmer; Sr. Developer; Expert,Data scientist,...,This is very important,This is very important,This is very important,I don't care about this,This is somewhat important,This is very important,This is somewhat important,Unrealistic expectations; Poor documentation; ...,To get help for my job; Beacause I love to learn,2229


# Problem:  Factor the Survey Matrix for Data Scientists using Nonnegative Matrix factorization

This is asking you to produce the matrix factorization of data scientists that you can.
The factors obtained will correspond to "classes" of data scientist.


To do this, first extract all rows with these job titles
into a <tt>DataScience</tt> table with about 2500 rows.

After eliminating as few columns and rows as possible,
perform nonnegative matrix factorization.

Finally, report the clearest factors you obtain this way.
These factors should identify types of "data scientist",
just as described in the Appendix of <b>Analyzing the Analyzers</b>.

In [18]:
from sklearn.decomposition import NMF # Sklearn's nonnegative matrix factorization
from sklearn import preprocessing # Preprocessing to change data's categorical variables to numerical

# Part 0:  Extract all "data scientists", using the 4 types of job titles

As mentioned above, you are supposed to include all 4 kinds of job titles in this model:
<ul><li>
<b>Business intelligence or data warehousing expert</b> (392 occurrences)
</li><li>
<b>Data scientist</b> (800 occurrences)
</li><li>
<b>Developer with a statistics or mathematics background</b>  (960)
</li><li>
<b>Machine learning developer</b> (385).
</li></ul>

In [19]:
DataScience = JobSurvey.copy() # Copy dataframe to manipulate without changing the original

# Extract a subset of the data involving data science
df1 = DataScience[ DataScience.occupation == 'Business intelligence or data warehousing expert' ]
df2 = DataScience[ DataScience.occupation == 'Data scientist' ]
df3 = DataScience[ DataScience.occupation == 'Developer with a statistics or mathematics background' ]
df4 = DataScience[ DataScience.occupation == 'Machine learning developer' ]

frames = [df1, df2, df3, df4]

DataScience = pd.concat(frames)

# Part 1: drop columns that have the most missing values for data scientists

In [20]:
#  Drop (or repair) columns from JobSurvey that have the most missing values.
#   The goal is to get a good matrix factorization.

In [21]:
###
### You can implement the method devised by your alert classmate Anna Merkoulovitch --
### She dropped all columns in which more than 10% of the values were missing.
###
### She told us that this got rid of some columns that didn't seem likely to help --
### including questions about aliens, cats and dogs -- and saved a lot of data.
###

In [22]:
#  Determine how many "data scientist" rows are left over after your careful dropping of columns.

In [23]:
Phillips_repairable_columns = 'self_identification,education,tech_do,tech_want,\
how_to_improve_interview_process,why_stack_overflow,dev_environment,\
developer_challenges,star_wars_vs_star_trek,new_job_value'.split(',') 
#  the .split(',') method turns this into a list of strings

for column in Phillips_repairable_columns:
    DataScience[column] = DataScience[column].fillna('')

max_big_mac_index = DataScience['big_mac_index'].max()
DataScience['big_mac_index'] = DataScience['big_mac_index'].fillna(max_big_mac_index)    

# Drop columns that have more than 10% of it's values as NaN
na_thresh = DataScience.shape[0] * 0.9        
DataScience = DataScience.dropna(thresh=na_thresh, axis=1)

# Drop variables that don't provide information about the data scientist
DataScience = DataScience.drop('Unnamed: 0', axis=1)
DataScience = DataScience.drop('jobId', axis=1)

# Part 2: drop all rows that still have null values (with .dropna)

In [24]:
DataScience = DataScience.dropna()

DataScience.shape

(1943, 34)

#  Part 3:  do the Nonnegative Matrix Factorization

In [25]:
#  consider for example:
#  http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

In [26]:
#  NMF factors a (n x p) dataset matrix X  into   X = W H
#   where W is (n x k) and H is (k x p), and k is the desired number of factors for NMF.
#   Thus the H matrix is a matrix of "factors" involving the p variables in X.
#   The factors are "weight vectors" that are interpreted like principal components.

In [27]:
le = preprocessing.LabelEncoder() # Preprocess data by changing all categorical variables to numerical
DS_trans = DataScience.apply(le.fit_transform) # Apply LabelEncoder to the data

In [28]:
model = NMF(n_components=4) # Build the NMF model
W_DS = model.fit_transform(DS_trans) # The features matrix W
H_DS = model.components_ # The coefficients matrix H (what we're interested in)

In [29]:
sig_val = 10 # If values in H are above this threshold, then we want to keep them
H_DS_df = pd.DataFrame(data=H_DS, columns=DataScience.columns.values) # Transform array into pandas dataframe
H_DS_df

,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,occupation_group,...,unit_testing,rep_range,visit_frequency,why_learn_new_tech,education,new_job_value,how_to_improve_interview_process,star_wars_vs_star_trek,developer_challenges,why_stack_overflow
0,0.132222,2.775095,0.584652,0.500926,0.078564,0.112342,0.046827,8.267387,0.082618,0.047142,...,0.131930,0.276001,0.116002,0.148791,6.041053,0.000000,5.709576,0.055125,0.000000,2.550139
1,0.184632,4.200371,0.645199,0.659550,0.132197,0.198054,0.079323,8.949906,0.145303,0.068715,...,0.198279,0.310022,0.135891,0.180159,8.863911,184.712698,11.424887,0.137815,0.000000,8.408735
2,0.119599,5.209185,0.714490,0.662538,0.175842,0.273127,0.072098,11.043672,0.057802,0.055000,...,0.096735,0.297747,0.155208,0.178029,9.144518,0.000000,3.874945,0.086372,0.000000,4.749302
3,0.131406,4.758243,1.130781,0.865284,0.229324,0.303737,0.099845,13.644362,0.182883,0.084762,...,0.226458,0.445409,0.156130,0.253183,16.213014,0.000000,15.412138,0.165127,202.942595,13.469958


In [36]:
# Individually sort each row in the dataframe into descending order

In [37]:
f1_df = H_DS_df.loc[[0]].sort_values(by=0, ascending=False, axis=1)
f1_df

,tech_want,dev_environment,self_identification,education,how_to_improve_interview_process,country,why_stack_overflow,big_mac_index,tech_do,salary_range,...,occupation,age_range,employment_status,aliens,star_wars_vs_star_trek,occupation_group,gender,dogs_vs_cats,new_job_value,developer_challenges
0,184.761201,16.345176,8.267387,6.041053,5.709576,2.775095,2.550139,1.040391,0.962194,0.584723,...,0.082618,0.078564,0.068288,0.059112,0.055125,0.047142,0.046827,0.031473,0.0,0.0


In [38]:
f2_df = H_DS_df.loc[[1]].sort_values(by=1, ascending=False, axis=1)
f2_df

,new_job_value,dev_environment,how_to_improve_interview_process,self_identification,education,why_stack_overflow,country,big_mac_index,salary_range,so_region,...,experience_range,age_range,aliens,gender,occupation_group,dogs_vs_cats,employment_status,tech_want,developer_challenges,tech_do
1,184.712698,28.095775,11.424887,8.949906,8.863911,8.408735,4.200371,1.580077,0.916371,0.65955,...,0.133578,0.132197,0.092167,0.079323,0.068715,0.065108,0.034601,0.0,0.0,0.0


In [39]:
f3_df = H_DS_df.loc[[2]].sort_values(by=2, ascending=False, axis=1)
f3_df

,tech_do,dev_environment,self_identification,education,country,why_stack_overflow,how_to_improve_interview_process,big_mac_index,salary_range,un_subregion,...,star_wars_vs_star_trek,dogs_vs_cats,gender,aliens,occupation,occupation_group,employment_status,new_job_value,tech_want,developer_challenges
2,217.247682,31.617635,11.043672,9.144518,5.209185,4.749302,3.874945,1.8585,0.975539,0.71449,...,0.086372,0.079682,0.072098,0.06184,0.057802,0.055,0.027843,0.0,0.0,0.0


In [40]:
f4_df = H_DS_df.loc[[3]].sort_values(by=3, ascending=False, axis=1)
f4_df

,developer_challenges,dev_environment,education,how_to_improve_interview_process,self_identification,why_stack_overflow,country,big_mac_index,salary_range,un_subregion,...,visit_frequency,collector,employment_status,aliens,gender,occupation_group,dogs_vs_cats,new_job_value,tech_want,tech_do
3,202.942595,45.096897,16.213014,15.412138,13.644362,13.469958,4.758243,1.683399,1.224363,1.130781,...,0.15613,0.131406,0.12605,0.11921,0.099845,0.084762,0.065859,0.0,0.0,0.0


In [41]:
# Puts the names of the columns that have a number over the significance threshold
def factor_values(df):
    f = [] # Create empty array to put significant column names into
    for i in range(0,(df.shape[1])-1): # For each column...
        if df.iloc[0][i] > sig_val: # If that columns value is greater than the significance value...
            f.append(df.columns[i]) # Add that column name into the factor array
        else:
            break # Repeat until you encounter a non-significant number and break because the array is sorted in descending order
    return(f) # Return the factor values in array form        

In [42]:
# Call factor value function for each of the factors
f1 = factor_values(f1_df)
f2 = factor_values(f2_df)
f3 = factor_values(f3_df)
f4 = factor_values(f4_df)

# Print the factor arrays
print("Factor 1: " + str(f1) + "\n")
print("Factor 2: " + str(f2) + "\n")
print("Factor 3: " + str(f3) + "\n")
print("Factor 4: " + str(f4))

Factor 1: ['tech_want', 'dev_environment']

Factor 2: ['new_job_value', 'dev_environment', 'how_to_improve_interview_process']

Factor 3: ['tech_do', 'dev_environment', 'self_identification']

Factor 4: ['developer_challenges', 'dev_environment', 'education', 'how_to_improve_interview_process', 'self_identification', 'why_stack_overflow']


## Insights

For reference, here are the relevant questions asked by the StackOverflow Tech Job Survey:

'tech_want' --- Which of the following languages or technologies do you WANT to work with this year? (select all that apply)

'dev_environment' --- What development environments do you use regularly? (select all that apply)

'new_job_value' --- When evaluating a new employment opportunity what's most important to you? (select up to 3)

'how_to_improve_interview_process' --- How can companies improve the interview process? (select up to 3)

'developer_challenges' --- What are the biggest challenges you face as a developer? (select up to 3)

'education' --- What amount of formal or professional programming training have you received? (select all that apply)

'self_identification' --- What do you consider yourself? (select all that apply)

'why_stack_overflow' --- Why do you use Stack Overflow? (select all that apply)

'tech_do' --- Which of the following languages or technologies have you done extensive development with in the last year?

NOTE: These are wild, and probably wildly inaccurate, characterizations gleaned from the data analysis. Please read with caution.

From the data, it appears that there are four distinct types of data scientists. In order:
    
1. The Enthusiast
This person enjoys coding, and displays a passion for learning new technology outside of their job. This is closest to "Data Creative".

2. The Go-Getter
This person cares deeply about their career, and the corporation they work for. They strive for both upward mobility and the success of the company. They are more likely to be in program/product manager roles or deal with the business side of data science. This is closest to "Data Businessperson".

3. The Specialist
This person has a favorite language and development environment. They are highly skilled and knowledgable in at least one area of data science. This is closest to "Data Developer".

4. The Problem Solver
This person likes to make meaningful discoveries with their data. They are interested in their work and how it furthers understanding of the world around them. They care about how they can make things "better" than they were before. This is closest to "Data Researcher".